In [17]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import fitz
from io import BytesIO
import string
import pandas as pd
import pickle
import time

In [18]:
k = 10

In [21]:
url = "https://www.sejm.gov.pl/Sejm10.nsf/agent.xsp?symbol=posglos&NrKadencji=10"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, "html.parser")

# Find all links containing "IdDnia" in the href attribute
links = soup.find_all("a", href=lambda href: href and "IdDnia" in href)


clean_links = []
for link in links:
    clean_links.append('https://www.sejm.gov.pl/sejm9.nsf/' + link['href'])

In [22]:
clean_links

['https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=2003',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=2005',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=2001',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1998',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1999',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1995',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1996',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1992',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1993',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1994',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1990',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1991',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1989',

In [23]:
clean_links_1 = []
for url in clean_links:    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", href=lambda href: href and "NrGlosowania" in href)

    for link in links:
        clean_links_1.append('https://www.sejm.gov.pl/sejm9.nsf/' + link['href'])

clean_links_1 = list(set(clean_links_1))  

In [24]:
clean_links_1

['https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=1&NrGlosowania=132',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=3&NrGlosowania=10',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=2&NrGlosowania=26',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=2&NrGlosowania=18',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=8&NrGlosowania=15',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=1&NrGlosowania=104',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=1&NrGlosowania=16',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=2&NrGlosowania=3',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=10&NrPosiedzenia=5&NrGlosowa

In [25]:
len(clean_links_1)

328

In [26]:
with open('clean_links_{k}.pkl', 'wb') as f:
    pickle.dump(clean_links_1, f)

In [27]:
vote_results_links = []
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
# headers={'User-Agent': 'Chrome/35.0.1916.47'}
r = requests.get(url, headers=headers)
for url in tqdm(clean_links_1):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    a = soup.find("a", text="Wyniki indywidualne")
    if a:
        vote_results_links.append(a['href'])
    else:
        print('sus url', url)
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████| 328/328 [08:49<00:00,  1.62s/it]


In [28]:
len(vote_results_links)

328

In [29]:
with open('vote_results_links_{k}.pkl', 'wb') as f:
    pickle.dump(vote_results_links, f)

In [4]:
vote_results_links = pickle.load(open(f'vote_results_links_{k}.pkl', 'rb'))

In [5]:
len(vote_results_links)

9388

https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=8&NrGlosowania=6

leads to 

https://orka.sejm.gov.pl/Glos9.nsf/nazwa/8_6/$file/GLOS_8_6.PDF

XDD

In [63]:
vote_results_links

['http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&1_132',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&3_10',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&2_26',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&2_18',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&8_15',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&1_104',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&1_16',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&2_3',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&5_4',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&8_12',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&6_18',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&2_23',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&1_135',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&7_6',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&2_4',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&6_17',
 'http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&7_5',
 'http://orka.se

In [61]:
# df = pd.DataFrame()

headers={'User-Agent': 'Chrome/35.0.1916.47'}
# headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

for pdf_url in tqdm(vote_results_links):
#     print(pdf_url)
    response = requests.get(url=pdf_url, headers=headers, timeout=120)
    on_fly_mem_obj = BytesIO(response.content)
    doc = fitz.open(stream=on_fly_mem_obj, filetype="pdf")
    
    all_text = ""
    for page in doc:
        all_text += '\n' + page.get_text()

    parties = re.findall("\n\w*.*\(\d+\)", all_text)
    if pdf_url == vote_results_links[2287]:
        parties.remove('\n(11)')
    clean_parties = [re.sub("\n", '', party) for party in parties]
    clean_parties = [re.sub("\(\d+\)", '', party).strip() for party in clean_parties]

    pattern = re.compile("\(\d+\)")
    deputies_per_club = [int(pattern.search(party).group(0)[1:-1]) for party in parties]

    xs = re.split("\n\w+.+\(\d+\)", all_text)
    
    # for the sake of my mental health I decided to ignore votings like:
    # https://orka.sejm.gov.pl/Glos9.nsf/nazwa/47_126/$file/Glos_47_126.pdf
    # because they have much different structure than 'normal' votings
    # and I have much better things to do
    if 'posiedzenie' not in xs[0].lower():
        print(pdf_url)
        continue

    posiedzenie_id = re.findall("posiedzenie \d+", xs[0].lower())[0]
    glosowanie_id = re.findall("sowanie nr \d+", xs[0])[0]
    data_time = re.findall("\d+.\d+.\d+\s\d+:\d+:\d+", xs[0])[0]
    posiedzenie_id = re.findall("\d+", posiedzenie_id)[0]
    glosowanie_id = re.findall("\d+", glosowanie_id)[0]

    for i in range(1, len(xs)):
        x = xs[i]
        x = re.sub("\n[\s\w.]+\s*-\s*\d+", '', x)
#         x = re.sub(r"\n\bPosiedzenie\b.+\n", '', x)
#         x = re.sub(r"\n\bPosiedzenie\b.+\n.*\bStrona.*\n", '', x)

        names = re.split("[za|pr\.|ng\.|ws\.]+", x)
        clean_names = []
        for name in names:
            name = re.sub("[\n\d]", '', name)
            name = re.sub("  ", ' ', name)
            name = name.strip()
            if len(name) > 0:
                clean_names.append(name)

        votes = re.findall("[za|pr\.|ng\.|ws\.]+", x)

        party = [clean_parties[i - 1]] * deputies_per_club[i - 1]

        df_sub = pd.DataFrame({
                'kadencja': k,
                'posiedzenie': posiedzenie_id,
                'numer': glosowanie_id,
                'data_godzina': data_time,
                'NazwiskoImie': clean_names[0:deputies_per_club[i - 1]], 
                'Glos': votes[0:deputies_per_club[i - 1]],
                'Klub': party
        })

        df = pd.concat([df, df_sub], ignore_index=True)
        
    if pdf_url == vote_results_links[2287]:
        party = ['Konfederacja'] * 11
        clean_names = ['KORWIN-MIKKE JANUSZ', 'BRAUN GRZEGORZ', 'TUDUJ KRZYSZTOF', 'KULESZA JAKUB', 
                       'SOŚNIERZ DOBROMIR', 'DZIAMBOR ARTUR', 'URBANIAK MICHAŁ', 'WINNICKI ROBERT', 
                       'BOSAK KRZYSZTOF', 'KAMIŃSKI KRYSTIAN',  'BERKOWICZ KONRAD']
        votes = ['za', 'pr.', 'ws.', 'ng.', 'za', 'pr.', 'ws.', 'ng.', 'pr.', 'ws.', 'ng']
        df_sub = pd.DataFrame({
                'kadencja': k,
                'posiedzenie': posiedzenie_id,
                'numer': glosowanie_id,
                'data_godzina': data_time,
                'NazwiskoImie': clean_names, 
                'Glos': votes,
                'Klub': party
        })
        df = pd.concat([df, df_sub], ignore_index=True)

 12%|█████████▎                                                                   | 859/7101 [15:32<1:35:16,  1.09it/s]

http://orka.sejm.gov.pl/Glos9.nsf/dok?OpenAgent&61_141


 53%|███████████████████████████████████████▏                                  | 3759/7101 [1:33:51<1:17:10,  1.39s/it]

http://orka.sejm.gov.pl/Glos9.nsf/dok?OpenAgent&31_18


 63%|██████████████████████████████████████████████▍                           | 4454/7101 [1:59:04<1:23:59,  1.90s/it]

http://orka.sejm.gov.pl/Glos9.nsf/dok?OpenAgent&34_1


 75%|█████████████████████████████████████████████████████████▎                  | 5358/7101 [2:34:01<52:00,  1.79s/it]

http://orka.sejm.gov.pl/Glos9.nsf/dok?OpenAgent&1_53


100%|████████████████████████████████████████████████████████████████████████████| 7101/7101 [3:53:17<00:00,  1.97s/it]


In [13]:
df2 = df.astype({
                'kadencja': 'int32',
                'posiedzenie': 'int32',
                'numer': 'int32',
                })

df2['data_godzina'] = pd.to_datetime(df['data_godzina'], format='mixed')

df2['vote_id'] = df2['kadencja'] * 10**6 + df2['posiedzenie'] * 10**3 + df2['numer']

In [76]:
df2['vote_id'].unique().size

9383

In [89]:
df2['Klub'].unique()

array(['PiS', 'KO', 'Lewica', 'PSL-Kukiz15', 'Konfederacja', 'niez.',
       'KP', 'Polska2050', 'Porozumienie', 'Kukiz15', 'LD', 'PS',
       'Wolnościowcy', 'PPS', 'SLD', 'PSL', 'WP'], dtype=object)

In [86]:
to_drop = df2[df2['vote_id'] == 9063060].index

In [87]:
df2 = df2.drop(to_drop)

In [91]:
df2['NazwiskoImie'].unique()

array(['ADAMCZYK ANDRZEJ', 'GOŁOJUCH KAZIMIERZ', 'MACIEREWICZ ANTONI',
       'SOIN AGNIESZKA', 'ANDRUSZKIEWICZ ADAM', 'GONCIARZ JAROSŁAW',
       'MAKSYMOWICZ WOJCIECH', 'SOŚNIERZ ANDRZEJ', 'ANDZEL WALDEMAR',
       'GONTARZ ROBERT', 'MALĄG MARLENA MAGDALENA', 'SÓJKA KATARZYNA',
       'ARDANOWSKI JAN KRZYSZTOF', 'GOSEK MARIUSZ', 'MALIK EWA',
       'SROKA MAGDALENA', 'ARENT IWONA', 'GOSIEWSKA MAŁGORZATA',
       'MAŁECKI JERZY', 'STACHOWIAK-RÓŻECKA MIROSŁAWA', 'AST MAREK',
       'GOWIN JAROSŁAW', 'MAŁECKI MACIEJ', 'STEFANIUK DARIUSZ',
       'BABALSKI ZBIGNIEW', 'GÓRSKA AGNIESZKA', 'MASŁOWSKA GABRIELA',
       'STRZAŁKA BEATA', 'BABINETZ PIOTR', 'GÓRSKI MACIEJ',
       'MATERNA JERZY', 'SUSKI MAREK', 'BARTOSIK RYSZARD',
       'GRÓBARCZYK MAREK', 'MATEUSIAK-PIELUCHA BEATA', 'SZAŁABAWKA ARTUR',
       'BARTUŚ BARBARA', 'GUT-MOSTOWY ANDRZEJ', 'MATUSIAK GRZEGORZ',
       'SZARAMA WOJCIECH', 'BASZKO MIECZYSŁAW', 'GWIAZDOWSKI KAZIMIERZ',
       'MATUSZEWSKI MAREK', 'SZCZUDŁO ALEKSANDRA',

In [93]:
df2 = df2.reset_index(drop=True)

In [96]:
df2 = df2.sort_values('vote_id')

In [98]:
df2.to_csv(f'scraped_data/kadencja_{k}.csv', index=False)

# term 10

In [64]:
link = ['https://orka.sejm.gov.pl/Glos10.nsf/nazwa/5_2/$file/5_2_20240207_101914.pdf']

In [74]:
df = pd.DataFrame()

headers={'User-Agent': 'Chrome/35.0.1916.47'}
# headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

for pdf_url in tqdm(vote_results_links):
# for pdf_url in link:
#     print(pdf_url)
    response = requests.get(url=pdf_url, headers=headers, timeout=120)
    on_fly_mem_obj = BytesIO(response.content)
    doc = fitz.open(stream=on_fly_mem_obj, filetype="pdf")
    
    all_text = ""
    for page in doc:
        all_text += '\n' + page.get_text()

    parties = re.findall("\n\w*.*\(\d+\)", all_text)
    clean_parties = [re.sub("\n", '', party) for party in parties]
    clean_parties = [re.sub("\(\d+\)", '', party).strip() for party in clean_parties]

    pattern = re.compile("\(\d+\)")
    deputies_per_club = [int(pattern.search(party).group(0)[1:-1]) for party in parties]

    xs = re.split("\n\w+.+\(\d+\)", all_text)
    if 'posiedzenie' not in xs[0].lower():
        print(pdf_url)
        continue

    posiedzenie_id = re.findall("posiedzenie \d+", xs[0].lower())[0]
    glosowanie_id = re.findall("sowanie nr \d+", xs[0])[0]
    data_time = re.findall("\d+.\d+.\d+\s\d+:\d+:\d+", xs[0])[0]
    posiedzenie_id = re.findall("\d+", posiedzenie_id)[0]
    glosowanie_id = re.findall("\d+", glosowanie_id)[0]

    for i in range(1, len(xs)):
        x = xs[i]
        x = re.sub("\n[\s\w.]+\s*-\s*\d+", '', x)
#         x = re.sub(r"\n\bPosiedzenie\b.+\n", '', x)
#         x = re.sub(r"\n\bPosiedzenie\b.+\n.*\bStrona.*\n", '', x)

        names = re.split(r"[za|pr.|ng.|ws.]+", x)
        clean_names = []
        for name in names:
            name = re.sub("[\n\d]", '', name)
            name = re.sub("  ", ' ', name)
            name = name.strip()
            if len(name) > 0:
                clean_names.append(name)

        votes = re.findall(r"[za|pr.|ng.|ws.]+", x)
        if len(set(votes)) > 4:
            print(pdf_url)
            print(set(votes))
            break
        party = [clean_parties[i - 1]] * deputies_per_club[i - 1]

        df_sub = pd.DataFrame({
                'kadencja': k,
                'posiedzenie': posiedzenie_id,
                'numer': glosowanie_id,
                'data_godzina': data_time,
                'NazwiskoImie': clean_names[0:deputies_per_club[i - 1]], 
                'Glos': votes[0:deputies_per_club[i - 1]],
                'Klub': party
        })

        df = pd.concat([df, df_sub], ignore_index=True)

  0%|▎                                                                                 | 1/328 [00:00<01:39,  3.29it/s]

http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&1_132
{'n', 's', 'nr', 'z', 'g', 'r', 'pr.', 'wan', 'na', 'ng.'}


  1%|▌                                                                                 | 2/328 [00:00<01:30,  3.58it/s]

http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&3_10
{'n', 's', 'nr', 'z', 'g', 'r', 'wan', 'na', 'za', 'ng.'}


  1%|▊                                                                                 | 3/328 [00:00<01:33,  3.48it/s]

...

http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&9_33
{'n', 's', 'nr', 'z', 'g', 'r', 'wan', 'na', 'za', 'ng.'}


 18%|██████████████▌                                                                  | 59/328 [00:20<01:20,  3.35it/s]

http://orka.sejm.gov.pl/Glos10.nsf/dok?OpenAgent&9_19
{'n', 's', 'nr', 'z', 'g', 'ng.', 'r', 'wan', 'na', 'za', 'ws.'}


 18%|██████████████▌                                                                  | 59/328 [00:21<01:37,  2.77it/s]


KeyboardInterrupt: 

In [66]:
df

,kadencja,posiedzenie,numer,data_godzina,NazwiskoImie,Glos,Klub
0,10,5,2,2024-02-07 10:19:14,ADAMCZYK ANDRZEJ,za,PiS
1,10,5,2,2024-02-07 10:19:14,ANDRUSZKIEWICZ ADAM,za,PiS
2,10,5,2,2024-02-07 10:19:14,ANDZEL WALDEMAR,za,PiS
3,10,5,2,2024-02-07 10:19:14,ARCISZEWSKA-MIELEWCZYK DOROTA,za,PiS
4,10,5,2,2024-02-07 10:19:14,ARDANOWSKI JAN KRZYSZTOF,za,PiS
...,...,...,...,...,...,...,...
453,10,5,2,2024-02-07 10:19:14,MULAWA KRZYSZTOF,ng.,Konfederacja
454,10,5,2,2024-02-07 10:19:14,WILK RYSZARD,ng.,Konfederacja
455,10,5,2,2024-02-07 10:19:14,JAKUBIAK MAREK,za,Kukiz15
456,10,5,2,2024-02-07 10:19:14,KUKIZ PAWEŁ,za,Kukiz15


In [67]:
df = df.astype({
                'kadencja': 'int32',
                'posiedzenie': 'int32',
                'numer': 'int32',
                })

df['data_godzina'] = pd.to_datetime(df['data_godzina'], format='mixed')

df['vote_id'] = df['kadencja'] * 10**6 + df['posiedzenie'] * 10**3 + df['numer']

In [68]:
len(vote_results_links)

328

In [69]:
df['vote_id'].unique().size

1

In [70]:
df['Glos'].unique()

array(['za', 'pr.', 'ng.'], dtype=object)

In [71]:
df['Klub'].unique()

array(['PiS', 'KO', 'Polska2050-TD', 'PSL-TD', 'Lewica', 'Konfederacja',
       'Kukiz15'], dtype=object)

In [72]:
df['NazwiskoImie'].unique()

array(['ADAMCZYK ANDRZEJ', 'ANDRUSZKIEWICZ ADAM', 'ANDZEL WALDEMAR',
       'ARCISZEWSKA-MIELEWCZYK DOROTA', 'ARDANOWSKI JAN KRZYSZTOF',
       'ARENT IWONA EWA', 'AST MAREK', 'BABINETZ PIOTR', 'BALUCH ANNA',
       'BARTUŚ BARBARA', 'BŁASZCZAK MARIUSZ', 'BOCHENEK RAFAŁ',
       'BOGUCKI JACEK', 'BOGUCKI ZBIGNIEW', 'BOROWIAK JOANNA',
       'BORYS-SZOPA BOŻENA', 'BUDA WALDEMAR', 'BURZYŃSKA LIDIA',
       'CHOJECKI ARTUR', 'CHOMA KAZIMIERZ BOGUSŁAW',
       'CHOROSIŃSKA DOMINIKA', 'CICHOLSKA ANNA EWA', 'CIESZYŃSKI JANUSZ',
       'CZARNECKI KRZYSZTOF', 'CZARNECKI WITOLD WOJCIECH',
       'CZARTORYSKI ARKADIUSZ', 'CZERWIŃSKA ANITA', 'DAJCZAK WŁADYSŁAW',
       'DĄBROWSKA-BANASZEK ANNA', 'DOLATA ZBIGNIEW',
       'DORYWALSKI BARTŁOMIEJ', 'DRABEK PRZEMYSŁAW', 'DUDA ELŻBIETA',
       'DWORCZYK MICHAŁ PAWEŁ', 'DZIEDZICZAK JAN MICHAŁ',
       'FILIPEK-SOBCZAK MAGDALENA', 'GAWRON ANDRZEJ', 'GEMBICKA ANNA',
       'GIŻYŃSKI SZYMON', 'GLIŃSKI PIOTR', 'GOLIŃSKA MAŁGORZATA',
       'GOŁOJUCH KAZIM

In [60]:
df[df['Glos'] == 'ag']

,kadencja,posiedzenie,numer,data_godzina,NazwiskoImie,Glos,Klub,vote_id
99857,10,1,1,2023-11-13 15:17:22,k,ag,KO,10001001
105795,10,2,11,2024-01-16 19:40:02,Ki,ag,KO,10002011


In [59]:
df = df.sort_values('vote_id')
df.to_csv(f'scraped_data/kadencja_{k}.csv', index=False)